In [ ]:
f = open('essay.txt', 'r')
t=f.read()
f.close()

In [ ]:
!pip install mistralai

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import numpy as np
import os

In [ ]:
!pip install mistralai langchain_experimental

In [ ]:
t

'The James Bond series focuses on the titular character, a fictional British Secret Service agent created in 1953 by writer Ian Fleming, who featured him in twelve novels and two short-story collections. Since Fleming\'s death in 1964, eight other authors have written authorised Bond novels or novelisations: Kingsley Amis, Christopher Wood, John Gardner, Raymond Benson, Sebastian Faulks, Jeffery Deaver, William Boyd, and Anthony Horowitz. The latest novel is With a Mind to Kill by Anthony Horowitz, published in May 2022. Additionally Charlie Higson wrote a series on a young James Bond, and Kate Westbrook wrote three novels based on the diaries of a recurring series character, Moneypenny.\n\nThe character—also known by the code number 007 (pronounced "double-oh-seven")—has also been adapted for television, radio, comic strip, video games and film. The films are one of the longest continually running film series and have grossed over US$7.04 billion in total at the box office, making it 

In [ ]:
import re

def split_text_into_chunks(text, chunk_size):
    # Split the text into sentences
    sentences = re.split(r'(?<=[.!?]) +', text)

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        # If adding the next sentence exceeds the chunk size, store the current chunk and start a new one
        if len(current_chunk) + len(sentence) + 1 > chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            # Add the sentence to the current chunk
            if current_chunk:
                current_chunk += " " + sentence
            else:
                current_chunk = sentence

    # Append any remaining text as the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    return chunks


chunk_size = 128
chunks = split_text_into_chunks(t, chunk_size)

print(f"Number of chunks: {len(chunks)}")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk}")



Number of chunks: 53
Chunk 1: 
Chunk 2: The James Bond series focuses on the titular character, a fictional British Secret Service agent created in 1953 by writer Ian Fleming, who featured him in twelve novels and two short-story collections.
Chunk 3: Since Fleming's death in 1964, eight other authors have written authorised Bond novels or novelisations: Kingsley Amis, Christopher Wood, John Gardner, Raymond Benson, Sebastian Faulks, Jeffery Deaver, William Boyd, and Anthony Horowitz.
Chunk 4: The latest novel is With a Mind to Kill by Anthony Horowitz, published in May 2022.
Chunk 5: Additionally Charlie Higson wrote a series on a young James Bond, and Kate Westbrook wrote three novels based on the diaries of a recurring series character, Moneypenny.

The character—also known by the code number 007 (pronounced "double-oh-seven")—has also been adapted for television, radio, comic strip, video games and film.
Chunk 6: The films are one of the longest continually running film series and 

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

# Initialize the Mistral AI client
client = MistralClient(api_key='p3T9odnj2XtqQLiMpAfi06r8ZBUrYEnB')


In [ ]:
def run_mistral(user_message, model="mistral-medium-latest"):
    SYS_PROMPT = f"""
    You are a network graph maker who extracts terms and their relations from a given context.
    You are provided with a context chunk (delimited by ```). Your task is to extract the ontology
    of terms mentioned in the given context. These terms should represent the key concepts as per the context.

    Thought 1: While traversing through each sentence, think about the key terms mentioned in it.
        - Terms may include objects, entities, locations, organizations, persons, conditions, acronyms, documents, services, concepts, etc.
        - Terms should be as atomistic as possible.

    Thought 2: Consider how these terms can be related to one another.
        - Terms mentioned in the same sentence or paragraph are typically related.
        - Terms can have multiple relationships with other terms.

    Thought 3: Identify the relationships between each pair of related terms.

    Format your output as a list of JSON objects. Each element of the list contains a pair of terms
    and the relationship between them, structured as follows:

    [
        {{
            "node_1": "A concept from the extracted ontology",
            "node_2": "A related concept from the extracted ontology",
            "edge": "Relationship between the two concepts (node_1 and node_2) in one or two words"
        }},
        {{
            "node_1": "...",
            "node_2": "...",
            "edge": "..."
        }},
        Also no extra text except the format i have specified
        ...
    ]

    My text begins from here -> {user_message}
    """

    # Assuming client and ChatMessage are correctly initialized and imported
    messages = [
        ChatMessage(role="user", content=SYS_PROMPT)
    ]

    # Assuming client.chat returns a response object with choices
    chat_response = client.chat(
        model=model,
        messages=messages
    )

    # Assuming chat_response.choices is a list and we retrieve the first choice
    if chat_response.choices:
        return chat_response.choices[0].message.content
    else:
        return "No response from the chat model."

In [ ]:
result1 = run_mistral(str(chunks[1]))

In [ ]:
type(result1)
result1

'[\n  {\n    "node_1": "James Bond series",\n    "node_2": "James Bond",\n    "edge": "focuses\\_on"\n  },\n  {\n    "node_1": "James Bond",\n    "node_2": "British Secret Service agent",\n    "edge": "is"\n  },\n  {\n    "node_1": "James Bond",\n    "node_2": "Ian Fleming",\n    "edge": "created\\_by"\n  },\n  {\n    "node_1": "Ian Fleming",\n    "node_2": "James Bond",\n    "edge": "featured\\_in",\n  },\n  {\n    "node_1": "Ian Fleming",\n    "node_2": "novels",\n    "edge": "wrote"\n  },\n  {\n    "node_1": "Ian Fleming",\n    "node_2": "short-story collections",\n    "edge": "wrote"\n  }\n]'

In [ ]:
import time


# Assuming chunks is a list of chunks to process


results = []  # To store all results

for i, chunk in enumerate(chunks):
    retry_attempts = 4  # Number of retry attempts
    attempt = 0
    success = False

    while attempt < retry_attempts:
        try:
            result = run_mistral(str(chunk))
            results.append(result)
            print(f"Chunk {i}: {result}")
            success = True
            break  # Exit retry loop on success
        except Exception as e:
            print(f"Chunk {i}: Error occurred - {e}")
            attempt += 1
            if attempt < retry_attempts:
                print(f"Retrying chunk {i} after {attempt} attempt(s)...")
                time.sleep(5)  # Adjust delay as needed

    if not success:
        print("Chunk {i}: Failed at , retrying ")

print("All chunks processed.")



Chunk 0: ```
In the field of machine learning, a model is trained on a dataset and then tested on new, unseen data. The goal is to make accurate predictions on this new data. Overfitting occurs when a model is too complex and fits the training data too closely, resulting in poor performance on new data. Underfitting occurs when a model is too simple and does not capture the underlying patterns in the data. A good model should strike a balance between complexity and simplicity to achieve good performance on both the training and new data. Cross-validation is a technique used to assess the performance of a model by splitting the dataset into multiple parts and training and testing the model on different combinations of these parts.
```
[
{
"node\_1": "machine learning",
"node\_2": "model",
"edge": "involves"
},
{
"node\_1": "model",
"node\_2": "dataset",
"edge": "trained\_on"
},
{
"node\_1": "model",
"node\_2": "new\_data",
"edge": "tested\_on"
},
{
"node\_1": "model",
"node\_2": "accura

In [ ]:

results

['```\nIn the field of machine learning, a model is trained on a dataset and then tested on new, unseen data. The goal is to make accurate predictions on this new data. Overfitting occurs when a model is too complex and fits the training data too closely, resulting in poor performance on new data. Underfitting occurs when a model is too simple and does not capture the underlying patterns in the data. A good model should strike a balance between complexity and simplicity to achieve good performance on both the training and new data. Cross-validation is a technique used to assess the performance of a model by splitting the dataset into multiple parts and training and testing the model on different combinations of these parts.\n```\n[\n{\n"node\\_1": "machine learning",\n"node\\_2": "model",\n"edge": "involves"\n},\n{\n"node\\_1": "model",\n"node\\_2": "dataset",\n"edge": "trained\\_on"\n},\n{\n"node\\_1": "model",\n"node\\_2": "new\\_data",\n"edge": "tested\\_on"\n},\n{\n"node\\_1": "mod

In [ ]:

type(results)

list

In [ ]:
results='''[
    {
        "node_1": "James Bond series",
        "node_2": "James Bond",
        "edge": "focuses_on"
    },
    {
        "node_1": "James Bond",
        "node_2": "British Secret Service agent",
        "edge": "is_a"
    },
    {
        "node_1": "James Bond",
        "node_2": "Ian Fleming",
        "edge": "created_by"
    },
    {
        "node_1": "Ian Fleming",
        "node_2": "novels",
        "edge": "featured_in"
    },
    {
        "node_1": "Ian Fleming",
        "node_2": "short-story collections",
        "edge": "featured_in"
    },
    {
        "node_1": "Fleming",
        "node_2": "Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "Fleming",
        "node_2": "Bond novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "Fleming",
        "node_2": "1964",
        "edge": "died_in"
    },
    {
        "node_1": "eight other authors",
        "node_2": "Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "eight other authors",
        "node_2": "Bond novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "Kingsley Amis",
        "node_2": "Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "Kingsley Amis",
        "node_2": "Bond novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "Christopher Wood",
        "node_2": "Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "Christopher Wood",
        "node_2": "Bond novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "John Gardner",
        "node_2": "Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "John Gardner",
        "node_2": "Bond novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "Raymond Benson",
        "node_2": "Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "Raymond Benson",
        "node_2": "Bond novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "Sebastian Faulks",
        "node_2": "Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "Sebastian Faulks",
        "node_2": "Bond novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "Jeffery Deaver",
        "node_2": "Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "Jeffery Deaver",
        "node_2": "Bond novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "William Boyd",
        "node_2": "Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "William Boyd",
        "node_2": "Bond novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "Anthony Horowitz",
        "node_2": "Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "Anthony Horowitz",
        "node_2": "Bond novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "The latest novel",
        "node_2": "With a Mind to Kill",
        "edge": "is"
    },
    {
        "node_1": "With a Mind to Kill",
        "node_2": "Anthony Horowitz",
        "edge": "by"
    },
    {
        "node_1": "With a Mind to Kill",
        "node_2": "May 2022",
        "edge": "published_in"
    },
    {
        "node_1": "Charlie Higson",
        "node_2": "young James Bond series",
        "edge": "wrote"
    },
    {
        "node_1": "Kate Westbrook",
        "node_2": "Moneypenny diaries novels",
        "edge": "wrote"
    },
    {
        "node_1": "James Bond",
        "node_2": "007",
        "edge": "also_known_as"
    },
    {
        "node_1": "James Bond",
        "node_2": "young James Bond series",
        "edge": "featured_in"
    },
    {
        "node_1": "Moneypenny",
        "node_2": "Moneypenny diaries novels",
        "edge": "featured_in"
    },
    {
        "node_1": "James Bond",
        "node_2": "television",
        "edge": "adapted_for"
    },
    {
        "node_1": "James Bond",
        "node_2": "radio",
        "edge": "adapted_for"
    },
    {
        "node_1": "James Bond",
        "node_2": "comic strip",
        "edge": "adapted_for"
    },
    {
        "node_1": "James Bond",
        "node_2": "video games",
        "edge": "adapted_for"
    },
    {
        "node_1": "James Bond",
        "node_2": "film",
        "edge": "adapted_for"
    },
    {
        "node_1": "films",
        "node_2": "film series",
        "edge": "belong_to"
    },
    {
        "node_1": "films",
        "node_2": "box office",
        "edge": "grossed_over"
    },
    {
        "node_1": "films",
        "node_2": "US$7.04 billion",
        "edge": "total_gross"
    },
    {
        "node_1": "film series",
        "node_2": "1962",
        "edge": "started_in"
    },
    {
        "node_1": "film series",
        "node_2": "fifth-highest-grossing",
        "edge": "ranked_as"
    },
    {
        "node_1": "film series",
        "node_2": "Dr. No",
        "edge": "started_with"
    },
    {
        "node_1": "Sean Connery",
        "node_2": "Bond",
        "edge": "portrays"
    },
    {
        "node_1": "Bond",
        "node_2": "Eon Productions",
        "edge": "series"
    },
    {
        "node_1": "Eon Productions",
        "node_2": "films",
        "edge": "produced"
    },
    {
        "node_1": "films",
        "node_2": "2021",
        "edge": "released_until"
    },
    {
        "node_1": "Bond film",
        "node_2": "No Time to Die (2021)",
        "edge": "is"
    },
    {
        "node_1": "No Time to Die (2021)",
        "node_2": "Daniel Craig",
        "edge": "stars"
    },
    {
        "node_1": "Daniel Craig",
        "node_2": "Bond",
        "edge": "portrays"
    },
    {
        "node_1": "Bond",
        "node_2": "Daniel Craig",
        "edge": "played_by"
    },
    {
        "node_1": "Daniel Craig",
        "node_2": "fifth",
        "edge": "has"
    },
    {
        "node_1": "Bond",
        "node_2": "Eon series",
        "edge": "is_part_of"
    },
    {
        "node_1": "Daniel Craig",
        "node_2": "sixth",
        "edge": "is"
    },
    {
        "node_1": "Bond",
        "node_2": "actor",
        "edge": "is_played_by"
    },
    {
        "node_1": "Bond film productions",
        "node_2": "Casino Royale (1967)",
        "edge": "includes"
    },
    {
        "node_1": "Casino Royale (1967)",
        "node_2": "David Niven",
        "edge": "starring"
    },
    {
        "node_1": "Bond film productions",
        "node_2": "Never Say Never Again (1983)",
        "edge": "includes"
    },
    {
        "node_1": "Bond",
        "node_2": "Scorpius",
        "edge": "published in"
    },
    {
        "node_1": "Bond",
        "node_2": "Win, Lose or Die",
        "edge": "published in"
    },
    {
        "node_1": "Bond",
        "node_2": "Licence to Kill",
        "edge": "novelisation of"
    },
    {
        "node_1": "Bond",
        "node_2": "Brokenclaw",
        "edge": "published in"
    },
    {
        "node_1": "Bond",
        "node_2": "The Man from Barbarossa",
        "edge": "published in"
    },
    {
        "node_1": "Bond",
        "node_2": "Death Is Forever",
        "edge": "published in"
    },
    {
        "node_1": "Bond",
        "node_2": "Never Send Flowers",
        "edge": "published in"
    },
    {
        "node_1": "Bond",
        "node_2": "SeaFire",
        "edge": "published in"
    },
    {
        "node_1": "Bond",
        "node_2": "GoldenEye",
        "edge": "novelisation of"
    },
    {
        "node_1": "Bond",
        "node_2": "Cold",
        "edge": "published in"
    },
    {
        "node_1": "Raymond Benson",
        "node_2": "Bond novels",
        "edge": "author of"
    },
    {
        "node_1": "Benson",
        "node_2": "The James Bond Bedside Companion",
        "edge": "authored"
    },
    {
        "node_1": "Benson",
        "node_2": "six Bond novels",
        "edge": "wrote"
    },
    {
        "node_1": "Benson",
        "node_2": "three novelisations",
        "edge": "wrote"
    },
    {
        "node_1": "Benson",
        "node_2": "three short stories",
        "edge": "wrote"
    },
    {
        "node_1": "Benson",
        "node_2": "Blast From the Past",
        "edge": "authored"
    },
    {
        "node_1": "Benson",
        "node_2": "Zero Minus Ten",
        "edge": "authored"
    },
    {
        "node_1": "Benson",
        "node_2": "Tomorrow Never Dies",
        "edge": "novelised"
    },
    {
        "node_1": "Benson",
        "node_2": "The Facts of Death",
        "edge": "authored"
    },
    {
        "node_1": "Benson",
        "node_2": "Midsummer Night's Doom",
        "edge": "authored"
    },
    {
        "node_1": "Benson",
        "node_2": "Live at Five",
        "edge": "authored"
    },
    {
        "node_1": "Benson",
        "node_2": "The World Is Not Enough",
        "edge": "novelised"
    },
    {
        "node_1": "Benson",
        "node_2": "High Time to Kill",
        "edge": "authored"
    },
    {
        "node_1": "Benson",
        "node_2": "DoubleShot",
        "edge": "authored"
    },
    {
        "node_1": "Benson",
        "node_2": "Never Dream of Dying",
        "edge": "authored"
    },
    {
        "node_1": "Benson",
        "node_2": "The Man with the Red Tattoo",
        "edge": "authored"
    },
    {
        "node_1": "Benson",
        "node_2": "Die Another Day",
        "edge": "novelised"
    },
    {
        "node_1": "Sebastian Faulks",
        "node_2": "Devil May Care",
        "edge": "authored"
    },
    {
        "node_1": "American writer Jeffery Deaver",
        "node_2": "Carte Blanche",
        "edge": "authored"
    },
    {
        "node_1": "William Boyd",
        "node_2": "Solo",
        "edge": "authored"
    },
    {
        "node_1": "Anthony Horowitz",
        "node_2": "Bond continuation novel",
        "edge": "to write"
    },
    {
        "node_1": "Bond continuation novel",
        "node_2": "Goldfinger",
        "edge": "follows events of"
    },
    {
        "node_1": "Trigger Mortis",
        "node_2": "Bond novel",
        "edge": "is_a"
    },
    {
        "node_1": "Trigger Mortis",
        "node_2": "Horowitz",
        "edge": "written_by"
    },
    {
        "node_1": "Trigger Mortis",
        "node_2": "8 September 2015",
        "edge": "released_on"
    },
    {
        "node_1": "Forever and a Day",
        "node_2": "Bond novel",
        "edge": "is_a"
    },
    {
        "node_1": "Forever and a Day",
        "node_2": "Horowitz",
        "edge": "written_by"
    },
    {
        "node_1": "Forever and a Day",
        "node_2": "Bond",
        "edge": "about"
    },
    {
        "node_1": "Forever and a Day",
        "node_2": "origin story",
        "edge": "tells"
    },
    {
        "node_1": "Bond",
        "node_2": "00 agent",
        "edge": "is_a"
    },
    {
        "node_1": "Forever and a Day",
        "node_2": "Casino Royale",
        "edge": "precedes"
    },
    {
        "node_1": "novel",
        "node_2": "unpublished material from Fleming",
        "edge": "based_on"
    },
    {
        "node_1": "novel",
        "node_2": "release",
        "edge": "announced_on"
    },
    {
        "node_1": "Horowitz's third Bond novel",
        "node_2": "publication",
        "edge": "occurred_on"
    },
    {
        "node_1": "Charlie Higson's first adult Bond novel",
        "node_2": "publication",
        "edge": "occurred_on"
    },
    {
        "node_1": "Charlie Higson's first adult Bond novel",
        "node_2": "Coronation of King Charles III",
        "edge": "celebrates"
    },
    {
        "node_1": "Charlie Higson's first adult Bond novel",
        "node_2": "National Literacy Trust",
        "edge": "supports"
    },
    {
        "node_1": "Devil May Care",
        "node_2": "release",
        "edge": "occurred_in"
    },
    {
        "node_1": "Carte Blanche",
        "node_2": "release",
        "edge": "occurred_in"
    },
    {
        "node_1": "Solo",
        "node_2": "release",
        "edge": "occurred_in"
    },
    {
        "node_1": "Trigger Mortis",
        "node_2": "release",
        "edge": "occurred_in"
    },
    {
        "node_1": "Forever and a Day",
        "node_2": "release",
        "edge": "occurred_in"
    },
    {
        "node_1": "With a Mind to Kill",
        "node_2": "release",
        "edge": "occurred_in"
    },
    {
        "node_1": "On His Majesty's Secret Service",
        "node_2": "publication",
        "edge": "occurred_on"
    },
    {
        "node_1": "Young Bond series",
        "node_2": "Charlie Higson",
        "edge": "started_by"
    },
    {
        "node_1": "Young Bond series",
        "node_2": "novels",
        "edge": "includes"
    },
    {
        "node_1": "SilverFin",
        "node_2": "graphic novel",
        "edge": "adapted_into"
    },
    {
        "node_1": "Young Bond series",
        "node_2": "Stephen Cole",
        "edge": "continued_by"
    },
    {
        "node_1": "The Moneypenny Diaries",
        "node_2": "novels",
        "edge": "comprises"
    },
    {
        "node_1": "The Moneypenny Diaries",
        "node_2": "Miss Moneypenny",
        "edge": "features"
    },
    {
        "node_1": "Samantha Weinberg",
        "node_2": "author",
        "edge": "is_a"
    },
    {
        "node_1": "Samantha Weinberg",
        "node_2": "Kate Westbrook",
        "edge": "pseudonym"
    }

,
    {
        "node_1": "Cubby Broccoli",
        "node_2": "Dr. No",
        "edge": "released"
    },
    {
        "node_1": "Dr. No",
        "node_2": "Ian Fleming",
        "edge": "novel_by"
    },
    {
        "node_1": "Dr. No",
        "node_2": "cinema adaptation",
        "edge": "is_a"
    },
    {
        "node_1": "Dr. No",
        "node_2": "Sean Connery",
        "edge": "starring"
    },
    {
        "node_1": "Sean Connery",
        "node_2": "James Bond",
        "edge": "portraying"
    },
    {
        "node_1": "No",
        "node_2": "1962",
        "edge": "released_in"
    },
    {
        "node_1": "No",
        "node_2": "1958 novel",
        "edge": "based_on"
    },
    {
        "node_1": "Sean Connery",
        "node_2": "007",
        "edge": "portrayed_as"
    },
    {
        "node_1": "Sean Connery",
        "node_2": "No",
        "edge": "starred_in"
    },
    {
        "node_1": "You Only Live Twice",
        "node_2": "1967",
        "edge": "released_in"
    },
    {
        "node_1": "Sean Connery",
        "node_2": "You Only Live Twice",
        "edge": "starred_in"
    },
    {
        "node_1": "George Lazenby",
        "node_2": "On Her Majesty's Secret Service",
        "edge": "starred_in"
    },
    {
        "node_1": "On Her Majesty's Secret Service",
        "node_2": "1969",
        "edge": "released_in"
    },
    {
        "node_1": "George Lazenby",
        "node_2": "007",
        "edge": "portrayed_as"
    },
    {
        "node_1": "Diamonds Are Forever",
        "node_2": "Sean Connery",
        "edge": "starred_in"
    },
    {
        "node_1": "Diamonds Are Forever",
        "node_2": "Eon-produced film",
        "edge": "is_a"
    },
    {
        "node_1": "Sean Connery",
        "node_2": "role",
        "edge": "left_after"
    },
    {
        "node_1": "Roger Moore",
        "node_2": "007",
        "edge": "appointed_as"
    },
    {
        "node_1": "Roger Moore",
        "node_2": "Live and Let Die",
        "edge": "starred_in"
    },
    {
        "node_1": "Live and Let Die",
        "node_2": "1973",
        "edge": "released_in"
    },
    {
        "node_1": "Bond",
        "node_2": "He",
        "edge": "played"
    },
    {
        "node_1": "He",
        "node_2": "Bond",
        "edge": "played"
    },
    {
        "node_1": "He",
        "node_2": "Timothy Dalton",
        "edge": "replaced_by"
    },
    {
        "node_1": "Timothy Dalton",
        "node_2": "He",
        "edge": "replaced"
    },
    {
        "node_1": "Bond",
        "node_2": "six times",
        "edge": "played"
    },
    {
        "node_1": "six times",
        "node_2": "Bond",
        "edge": "played"
    },
    {
        "node_1": "He",
        "node_2": "twelve years",
        "edge": "over"
    },
    {
        "node_1": "twelve years",
        "node_2": "He",
        "edge": "over"
    },
    {
        "node_1": "Timothy Dalton",
        "node_2": "two films",
        "edge": "for"
    },
    {
        "node_1": "two films",
        "node_2": "Timothy Dalton",
        "edge": "for"
    },
    {
        "node_1": "Six-year hiatus",
        "node_2": "Eon's productions",
        "edge": "disrupted"
    },
    {
        "node_1": "Legal wrangle",
        "node_2": "Eon's productions",
        "edge": "threatened"
    },
    {
        "node_1": "Pierce Brosnan",
        "node_2": "Bond",
        "edge": "cast_as"
    },
    {
        "node_1": "Pierce Brosnan",
        "node_2": "GoldenEye",
        "edge": "cast_in"
    },
    {
        "node_1": "Pierce Brosnan",
        "node_2": "Bond films",
        "edge": "featured_in"
    },
    {
        "node_1": "Bond films",
        "node_2": "2002",
        "edge": "through"
    },
    {
        "node_1": "Daniel Craig",
        "node_2": "Casino Royale (2006)",
        "edge": "played_in"
    },
    {
        "node_1": "Casino Royale (2006)",
        "node_2": "film series",
        "edge": "rebooted"
    },
    {
        "node_1": "Daniel Craig",
        "node_2": "film series",
        "edge": "appeared_in"
    },
    {
        "node_1": "film series",
        "node_2": "2006",
        "edge": "started_in"
    },
    {
        "node_1": "film series",
        "node_2": "$7 billion",
        "edge": "grossed"
    },
    {
        "node_1": "film series",
        "node_2": "fifth-highest-grossing",
        "edge": "is"
    }]'''
type(results)


str

In [ ]:
import json
import pandas as pd
data = json.loads(results)

# Convert to DataFrame
df = pd.DataFrame(data)


print("DataFrame:")
print(df)

DataFrame:
                   node_1                        node_2         edge
0       James Bond series                    James Bond   focuses_on
1              James Bond  British Secret Service agent         is_a
2              James Bond                   Ian Fleming   created_by
3             Ian Fleming                        novels  featured_in
4             Ian Fleming       short-story collections  featured_in
..                    ...                           ...          ...
159  Casino Royale (2006)                   film series     rebooted
160          Daniel Craig                   film series  appeared_in
161           film series                          2006   started_in
162           film series                    $7 billion      grossed
163           film series        fifth-highest-grossing           is

[164 rows x 3 columns]


In [ ]:
df.to_csv("concepts.csv", sep="|", index=False)

In [ ]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade pyvis


In [ ]:
data = json.loads(results)

In [ ]:
import json
from neo4j import GraphDatabase

# Neo4j connection details
uri = "neo4j+s://0a5f0844.databases.neo4j.io"  # Replace with your Neo4j instance URI
user = "neo4j"                 # Replace with your Neo4j username
password = "cDWOTr50HdcjUsPk4kY_5bzlUwrNiAs1C2u8aVzl9Wc"          # Replace with your Neo4j password

# Connect to Neo4j
driver = GraphDatabase.driver(uri, auth=(user, password))

# JSON data as a string

# Convert the string to JSON
data = json.loads(results)

# Function to create nodes and relationships
def create_graph(tx, data):
    # Set to keep track of created nodes to avoid duplicates
    created_nodes = set()

    for item in data:
        node_1 = item['node_1']
        node_2 = item['node_2']
        edge = item['edge']

        # Create node_1 if it doesn't exist
        if node_1 not in created_nodes:
            tx.run("MERGE (n1:Entity {name: $name})", name=node_1)
            created_nodes.add(node_1)

        # Create node_2 if it doesn't exist
        if node_2 not in created_nodes:
            tx.run("MERGE (n2:Entity {name: $name})", name=node_2)
            created_nodes.add(node_2)

        # Create relationship
        query = f"""
            MATCH (n1:Entity {{name: $node_1}}), (n2:Entity {{name: $node_2}})
            MERGE (n1)-[:`{edge}`]->(n2)
        """

# Create nodes and relationships within a session
with driver.session() as session:
    session.write_transaction(create_graph, data)

# Close the driver
driver.close()


<ipython-input-9-42f0b4fb4976>:45: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_graph, data)
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information